In [12]:
#!/usr/local/bin/python
# -*- coding: utf-8 -*-
# Copyright (c) 2017 Mikhail Shilov

# Обеспечим совместимость с Python 2 и 3
# pip install future
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

# импортируем Pandas и Numpy
import numpy as np
import pandas as pd
import seaborn
import plotly.plotly as py
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected = True)

cnt_day =  70 # количество дней для просмотра

#По умолчанию Pandas выводит всего 20 столбцов и 60 строк. Если нужно более ...
pd.set_option('display.max_columns', 65)
pd.set_option('display.max_rows', 65)

path = "comparative_analysis.csv"  
data = pd.read_csv(path, delimiter=";")
data['date'] =  pd.to_datetime(data.date, format="%d.%m.%Y")
data['facebook_prophet'] = np.log(data['facebook_prophet'])
data['excel'] = np.log(data['excel'])
data['calc'] = np.log(data['calc'])
data['fact'] = np.log(data['fact'])

data = data[0:cnt_day][['date', 'facebook_prophet', 'excel','calc','fact']]
data.set_index(data['date'],inplace=True)

data.head()
#print (data)

,date,facebook_prophet,excel,calc,fact
date,,,,,
2017-07-29,2017-07-29,30.126481,30.297378,30.126481,30.111644
2017-07-30,2017-07-30,30.139244,30.297591,30.139244,30.316897
2017-07-31,2017-07-31,30.150905,30.297804,30.150905,30.277179
2017-08-01,2017-08-01,30.161138,30.298017,30.161138,30.348650
2017-08-02,2017-08-02,30.171779,30.298230,30.171779,30.311933


In [13]:
#


def show_forecast(data):

    # Facebook Prophet
    facebook_prophet = go.Scatter(
        name='Facebook Prophet',
        x=data.index,
        y=data.facebook_prophet,
        mode='lines',
        line=dict(color='#17BECF'),
        opacity = 0.8,
    )
    
      # excel
    excel = go.Scatter(
        name='Excel',
        x=data.index,
        y=data.excel,
        marker=dict(color="#7F7F7F"),
        mode='lines',
        opacity = 0.8,
    )
    
    
     # calc
    calc = go.Scatter(
        name='Calc',
        x=data.index,
        y=data.calc,
        marker=dict(color="#71B33C"),
        mode='lines',
        opacity = 0.8,
    )
    
  
    # fact
    fact = go.Scatter(
        name='Fact',
        x=data.index,
        y=data.fact,
        marker=dict(color="8080F0"),
        mode='lines',
         opacity = 0.8,
    )

    
    data = [facebook_prophet, fact, excel, calc]

    layout = go.Layout(
       
        yaxis=dict(title='Хэшрейт сети DASH'),
        title='Сравнение алгоритмов прогнозирования с фактическим хэшрейтом ',
        showlegend = True,
        )
    
    
    layout.update(dict(annotations=[
        go.Annotation(text="Прогноз калькулятора пока <br> совпадает с Facebook Prophet ", x='2017-08-21', y=30.35),
        go.Annotation(text="Здесь зажжужат первые асики ", x='2017-09-10', y=30.49)
    ]))
    
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

show_forecast(data)

In [14]:
#Оценка качества Prophet
data['e_prophet'] = data['fact'] - data['facebook_prophet']
data['p_prophet'] = 100*data['e_prophet']/data['fact']
np.mean(abs(data[0:cnt_day]['p_prophet'])), np.mean(abs(data[0:cnt_day]['e_prophet']))


(2.4603520010394773, 0.7833366333727286)

In [15]:
#Оценка качества Excel
data['e_excel'] = data['fact'] - data['excel']
data['p_excel'] = 100*data['e_excel']/data['fact']
np.mean(abs(data[0:cnt_day]['p_excel'])), np.mean(abs(data[0:cnt_day]['e_excel']))

(2.841198427889152, 0.9049642851552611)

In [16]:
#Оценка качества calc
data['e_calc'] = data['fact'] - data['calc']
data['p_calc'] = 100*data['e_calc']/data['fact']
np.mean(abs(data[0:cnt_day]['p_calc'])), np.mean(abs(data[0:cnt_day]['e_calc']))

(1.7036842991479242, 0.5387275924693694)